In [ ]:
# %pip install python-dotenv pandas pinecone langchain langchain-openai langchain-pinecone scikit-learn matplotlib


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_LLM_MODEL = os.getenv("OPENAI_LLM_MODEL")  # 'gpt-4o-mini'
OPENAI_EMBEDDING_MODEL = os.getenv("OPENAI_EMBEDDING_MODEL")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_REGION = os.getenv("PINECONE_INDEX_REGION")
PINECONE_INDEX_CLOUD = os.getenv("PINECONE_INDEX_CLOUD")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")  # 'ir'
PINECONE_INDEX_METRIC = os.getenv("PINECONE_INDEX_METRIC")
PINECONE_INDEX_DIMENSION = int(os.getenv("PINECONE_INDEX_DIMENSION"))

# 압축 인덱스 이름
COMPRESSED_INDEX_NAME = f"{PINECONE_INDEX_NAME}-compressed"

print(f"환경 변수 로딩 완료 : {PINECONE_INDEX_NAME}, {PINECONE_INDEX_REGION}, {PINECONE_INDEX_CLOUD}")

환경 변수 로딩 완료 : ir-embeddings, us-east-1, aws


In [3]:
import pandas as pd

# queries_meta.csv 로드
queries_df = pd.read_csv("../../datas/queries_meta.csv")
print(f"질의 수: {len(queries_df)}")
queries_df.head()

질의 수: 30


,query_id,query_text,relevant_doc_ids
0,Q01,저자 김민수의 문서를 모두 보여줘,D1=1;D11=1;D21=1
1,Q02,저자 이영희의 문서를 모두 보여줘,D2=1;D12=1;D22=1
2,Q03,저자 박지훈의 문서를 모두 보여줘,D3=1;D13=1;D23=1
3,Q04,저자 최수정의 문서를 모두 보여줘,D4=1;D14=1;D24=1
4,Q05,저자 정우성의 문서를 모두 보여줘,D5=1;D15=1;D25=1


In [4]:
from pinecone import Pinecone, ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# Pinecone 클라이언트 연결
pc = Pinecone(api_key=PINECONE_API_KEY)

# 인덱스가 없으면 생성
PINECONE_INDEX_NAME = "ir-embeddings-meta"
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=PINECONE_INDEX_DIMENSION,
        metric=PINECONE_INDEX_METRIC,
        spec=ServerlessSpec(region=PINECONE_INDEX_REGION, cloud=PINECONE_INDEX_CLOUD)
    )
index = pc.Index(PINECONE_INDEX_NAME)

# 임베딩 모델 생성
embedding_model = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL, openai_api_key=os.getenv("OPENAI_API_KEY"))
# Pinecone 벡터 스토어 설정
vector_store = PineconeVectorStore(index_name=PINECONE_INDEX_NAME, embedding=embedding_model)

print("Pinecone 및 벡터 스토어 준비 완료")

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pinecone 및 벡터 스토어 준비 완료


In [5]:
from langchain_openai import ChatOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

# ChatOpenAI 인스턴스 생성 (LLM: OpenAI Chat 모델 사용)
chat_model = ChatOpenAI(
    model_name=OPENAI_LLM_MODEL,
    openai_api_key=OPENAI_API_KEY,
    temperature=0.0
)

# 메타데이터 필드 정보 설정
metadata_field_info = [
    AttributeInfo(name='author', type='string', description='문서를 작성한 저자 이름'),
    AttributeInfo(name='category', type='list[string]', description='문서의 분류 리스트'),
    AttributeInfo(name='text', type='string', description='문서 본문 요약 또는 대표 텍스트')
]

# SelfQueryRetriever 생성 (LLM을 통해 질의 분석 및 벡터스토어 검색)
self_query_retriever = SelfQueryRetriever.from_llm(
    llm=chat_model,
    vectorstore=vector_store,
    document_contents="",          # 문서 내용 필드 (여기서는 생략 또는 기본값 사용)
    metadata_field_info=metadata_field_info,
    verbose=True
)

print("Self-Query Retriever 구성 완료")

Self-Query Retriever 구성 완료


In [6]:
import numpy as np

def parse_relevant(relevant_str):
    pairs = relevant_str.split(';')
    rel_dict = {}
    for pair in pairs:
        doc_id, grade = pair.split('=')
        rel_dict[doc_id] = int(grade)
    return rel_dict

def compute_metrics(predicted, relevant_dict, k=5):
    # Precision@k: 상위 k개 예측 중 정답 수 (hits) / k
    hits = sum(1 for doc in predicted[:k] if doc in relevant_dict)
    precision = hits / k
    # Recall@k: 정답 중 상위 k에 들어온 비율
    total_relevant = len(relevant_dict)
    recall = hits / total_relevant if total_relevant > 0 else 0
    # Reciprocal Rank: 첫 번째로 맞힌 정답의 역순위
    rr = 0
    for idx, doc in enumerate(predicted):
        if doc in relevant_dict:
            rr = 1 / (idx + 1)  # (idx는 0부터 시작하므로 +1이 실제 순위)
            break
    # Average Precision: 정답을 찾을 때마다의 precision의 평균
    num_correct = 0
    precisions = []
    for i, doc in enumerate(predicted[:k]):
        if doc in relevant_dict:
            num_correct += 1
            precisions.append(num_correct / (i + 1))
    ap = np.mean(precisions) if precisions else 0
    return precision, recall, rr, ap

def evaluate_all(results_dict, queries_df, k=5):
    prec_list, rec_list, rr_list, ap_list = [], [], [], []
    for idx, row in queries_df.iterrows():
        qid = row['query_id']
        relevant = parse_relevant(row['relevant_doc_ids'])
        predicted = results_dict[qid]
        p, r, rr, ap = compute_metrics(predicted, relevant, k)
        prec_list.append(p)
        rec_list.append(r)
        rr_list.append(rr)
        ap_list.append(ap)
    return {
        'P@5': np.mean(prec_list),
        'R@5': np.mean(rec_list),
        'MRR': np.mean(rr_list),
        'MAP': np.mean(ap_list)
    }

In [7]:
self_query_retriever.invoke('저자 이영희의 문서를 모두 보여줘')

[Document(id='b0887060-84d6-4d28-b1c6-211b230845c4', metadata={'author': '이영희', 'category': ['기술', '프로그래밍', '소프트웨어 개발'], 'doc_id': 'D22'}, page_content='싱글톤(Singleton) 패턴은 클래스의 인스턴스를 한 개만 생성하도록 보장하며, 전역 접근 지점을 제공하는 디자인 패턴입니다. Java 예제에서 `private static` 인스턴스 변수와 `getInstance()` 메서드를 사용해 단일 인스턴스를 반환합니다. 멀티스레드 환경에서는 `synchronized` 블록 또는 `volatile` 키워드를 활용해 안전을 보장해야 합니다.'),
 Document(id='807a33e2-341f-4cd1-903d-e2971fc3284a', metadata={'author': '이영희', 'category': ['기술', '프로그래밍', '소프트웨어 개발'], 'doc_id': 'D22'}, page_content='싱글톤(Singleton) 패턴은 클래스의 인스턴스를 한 개만 생성하도록 보장하며, 전역 접근 지점을 제공하는 디자인 패턴입니다. Java 예제에서 `private static` 인스턴스 변수와 `getInstance()` 메서드를 사용해 단일 인스턴스를 반환합니다. 멀티스레드 환경에서는 `synchronized` 블록 또는 `volatile` 키워드를 활용해 안전을 보장해야 합니다.'),
 Document(id='7767e82e-7a56-43dd-9767-e1f5939ad21e', metadata={'author': '이영희', 'category': ['여행', '레저'], 'doc_id': 'D12'}, page_content='서울 근교에서 당일치기로 다녀올 만한 여행지로는 가평 쁘띠프랑스, 남양주 수종사, 양평 두물머리, 용인 에버랜드 등이 있습니다. 기차·버스 노선이 잘 발달되어 있어 대중교통으로 이동이 편리하며, 차가 있다면 

In [ ]:
dense_results = {}
selfquery_results = {}

for idx, row in queries_df.iterrows():
    qid = row['query_id']
    query_text = row['query_text']
    # 1) Dense Retrieval: 임베딩 유사도 검색
    docs_dense = vector_store.similarity_search(query_text, k=5)
    dense_results[qid] = [doc.metadata['doc_id'] for doc in docs_dense]
    # 2) Self-Query Retriever: LLM 기반 검색
    docs_self = self_query_retriever.invoke(query_text)
    selfquery_results[qid] = [doc.metadata['doc_id'] for doc in docs_self[:5]]

print("검색 결과 수집 완료")

In [ ]:
import pandas as pd

dense_metrics = evaluate_all(dense_results, queries_df, k=5)
selfquery_metrics = evaluate_all(selfquery_results, queries_df, k=5)

df_metrics = pd.DataFrame({
    'Metric': ['P@5', 'R@5', 'MRR', 'MAP'],
    'Dense': [dense_metrics['P@5'], dense_metrics['R@5'], dense_metrics['MRR'], dense_metrics['MAP']],
    'SelfQuery': [selfquery_metrics['P@5'], selfquery_metrics['R@5'], selfquery_metrics['MRR'], selfquery_metrics['MAP']]
})
df_metrics